# Project : Data Extraction from Pin Code Site

> # Author : KRASHNA GURME
> # Date : 03/03/2021

   > * Extracting data from http://pincode.india-server.com/ 
   > * Here, we're extracting variables/columns like State, City, Post Office and Pin Code. Perform this operation only for Assam and Bihar states.


In [7]:
# Importing necessary packages and modules 

from bs4 import BeautifulSoup as bs
import urllib
import re
import requests
from urllib.request import urlopen

# Function to get anchor links 
def getLinks(url):
    html_page = urlopen(url)
    soup = bs(html_page)
    links = []

    for link in soup.findAll('a', attrs={'href': re.compile("^http://")}):
        links.append(link.get('href'))

    return links

# Stroring all links of State in List variable
allListStateLinks = getLinks('http://pincode.india-server.com/')

In [8]:
# Create a array fo rdynamic state input

stateList = ["assam","bihar"]
dictListofStateLinks = {}

# Get all State Links 
for eachStateName in stateList:
    for items in allListStateLinks:
        if eachStateName in items:
            stateLink = items
            break;
    dictListofStateLinks[eachStateName] = stateLink
#print(dictListofStateLinks)

In [9]:
# Get all District respective to each State simultaneously
dictAllDistrictLinks = {}
for eachState in dictListofStateLinks:    
    dictAllDistrictLinks[eachState] = getLinks(dictListofStateLinks[eachState])
                                                                    
#print(dictAllDistrictLinks)

In [10]:
# print(dictAllDistrictLinks) last output 

# Remove other links which are not of the respective State-District City Link
wordCities = 'cities'
allDistrictCities = {}

for district in dictAllDistrictLinks:
    allDistrictlink = []
    for items in dictAllDistrictLinks[district]:
        if wordCities in items:
            allDistrictlink.append(items)
    allDistrictCities[district]=allDistrictlink
    
#print(allDistrictCities)

In [11]:
# last input >> allDistrictCities
# Get Post Names and Pincodes with respective District of parent State
alldictStateData = {}
postOfficeWord = 'postoffice'

# Iterating throu' all the Post Links and trimming the rest part so that user get only PostName and POstal Code respectively 
for districtItem in allDistrictCities:
    dictStateData = {}
    for items_nn in allDistrictCities[districtItem]:   # district thro
        postOfficeList =  getLinks(items_nn)  #
        cityPostLink = []
        split_str_1 = items_nn.split("/")
        
        for each_item in postOfficeList:
            if postOfficeWord in each_item:
                split_str = each_item.split("/")
                #print(split_str)
                cityPostLink.append(split_str[4])
       
        dictStateData[split_str_1[4]] = list(cityPostLink)  
    alldictStateData[districtItem] = dictStateData    
#print(alldictStateData)

In [12]:
# alldictStateData >> last op

# Print Result in format of State:District::City:::Pin Code(JSON Format-dict)
final_result = {}

# Output Formatting Logic as a JSON Format in Nested Dictionary
for eachState_Name in alldictStateData:
    result = []
    for key in alldictStateData[eachState_Name]:
        for i in range(len(alldictStateData[eachState_Name][key])-1):
            each_post = alldictStateData[eachState_Name][key][i]
            postDetails = each_post.split("-") 
            lengthPost = len(postDetails)
            pincode = postDetails[len(postDetails)-1]
            Postoffice = postDetails[0]
            if lengthPost > 2:
                varUse = postDetails.pop(len(postDetails)-1)
                Postoffice = " ".join(postDetails)

            dictSorted = {}
            dictSorted["State"] = eachState_Name
            dictSorted["City"] = key                      
            dictSorted["Post-office"] = Postoffice
            dictSorted["Pin-code"] = pincode 
        result.append(dictSorted)    
    final_result[eachState_Name]=result
    
print(final_result)

{'assam': [{'State': 'assam', 'City': 'barpeta', 'Post-office': 'thakurbazar non delivery', 'Pin-code': '781301'}, {'State': 'assam', 'City': 'bongaigaon', 'Post-office': 'topgaon', 'Pin-code': '783384'}, {'State': 'assam', 'City': 'cachar', 'Post-office': 'uttarkrishnapur', 'Pin-code': '788006'}, {'State': 'assam', 'City': 'darrang', 'Post-office': 'thana udalguri', 'Pin-code': '784509'}, {'State': 'assam', 'City': 'dhemaji', 'Post-office': 'talem non delivery', 'Pin-code': '787061'}, {'State': 'assam', 'City': 'dhubri', 'Post-office': 'tokrerchara', 'Pin-code': '783334'}, {'State': 'assam', 'City': 'dibrugarh', 'Post-office': 'ushapur', 'Pin-code': '786614'}, {'State': 'assam', 'City': 'goalpara', 'Post-office': 'upartala', 'Pin-code': '783124'}, {'State': 'assam', 'City': 'golaghat', 'Post-office': 'upper lengtha', 'Pin-code': '785602'}, {'State': 'assam', 'City': 'hailakandi', 'Post-office': 'vetarband', 'Pin-code': '788736'}, {'State': 'assam', 'City': 'jorhat', 'Post-office': 'ur

> Here, you could use a online JSON formatter for getting above data into more redable format. 
> * https://jsonformatter.curiousconcept.com/